Welcome to this example script of building electrification WH automation modeling. We will explain step by step what this script does using an example with two input files, one is the target file to be electrified, and the other one is the source file of electrified WH reference.

First step, install EPPY.

In [1]:
pip install eppy

Note: you may need to restart the kernel to use updated packages.


Install all packages that might be used

In [2]:
from eppy import modeleditor
from eppy.modeleditor import IDF
import pandas as pd
import os.path
import glob

Locate IDD file. In this case I saved IDD under the same folder with this notebook.

In [3]:
iddfile = 'Energy+.idd'
IDF.setiddname(iddfile)

Import two idf files. The target idf is the model to be electrified. The source idf is the model with the electrified WH system, regardless whether it's the same building type as the target.

IMPORTANT: Although the building types and the number of the plants of the target and source files do not have to be the same, the water heating distribution system needs to be aligned - so far, central WH system can only be replaced by another central WH system, and non-central system can only be replaced by another non-central system. A source file with a central WH system CANNOT be used to electrify a target file with a non-central WH system. It has to be one-on-one replacement.

In [4]:
#target file is DOE reference building of SFH
targetfname = 'US+MF+CZ2AWH+gasfurnace+slab+IECC_2006.idf'
targetidf = IDF(targetfname)

In [5]:
#source file is a SFH with HPWH
sourcefname = 'US+SF+CZ1AWHT+elecres+slab+IECC_2006_HPWH.idf'
sourceidf = IDF(sourcefname)

# Find target zones

We first identify zone served with DHW in the target model. This involve one manual step to rule out the unserved zone(s). The DHW serving zones will be used in the later step of WH electrification replacement.

In [6]:
#Number of zones in the target model
print(len(targetidf.idfobjects['Zone']))

20


In [7]:
#Name of zones, both conditioned and unconditioned, in the target model
zones=[]
for i in range(len(targetidf.idfobjects['Zone'])):
    zones.append(targetidf.idfobjects['Zone'][i].Name)

zones

['Breezeway',
 'living_unit1_FrontRow_BottomFloor',
 'living_unit2_FrontRow_BottomFloor',
 'living_unit3_FrontRow_BottomFloor',
 'living_unit1_BackRow_BottomFloor',
 'living_unit2_BackRow_BottomFloor',
 'living_unit3_BackRow_BottomFloor',
 'living_unit1_FrontRow_MiddleFloor',
 'living_unit2_FrontRow_MiddleFloor',
 'living_unit3_FrontRow_MiddleFloor',
 'living_unit1_BackRow_MiddleFloor',
 'living_unit2_BackRow_MiddleFloor',
 'living_unit3_BackRow_MiddleFloor',
 'attic',
 'living_unit1_FrontRow_TopFloor',
 'living_unit2_FrontRow_TopFloor',
 'living_unit3_FrontRow_TopFloor',
 'living_unit1_BackRow_TopFloor',
 'living_unit2_BackRow_TopFloor',
 'living_unit3_BackRow_TopFloor']

In [8]:
#Rule out unconditioned zone(s) - this step needs manual observation.
zones.remove('attic')
zones.remove('Breezeway')
zones

['living_unit1_FrontRow_BottomFloor',
 'living_unit2_FrontRow_BottomFloor',
 'living_unit3_FrontRow_BottomFloor',
 'living_unit1_BackRow_BottomFloor',
 'living_unit2_BackRow_BottomFloor',
 'living_unit3_BackRow_BottomFloor',
 'living_unit1_FrontRow_MiddleFloor',
 'living_unit2_FrontRow_MiddleFloor',
 'living_unit3_FrontRow_MiddleFloor',
 'living_unit1_BackRow_MiddleFloor',
 'living_unit2_BackRow_MiddleFloor',
 'living_unit3_BackRow_MiddleFloor',
 'living_unit1_FrontRow_TopFloor',
 'living_unit2_FrontRow_TopFloor',
 'living_unit3_FrontRow_TopFloor',
 'living_unit1_BackRow_TopFloor',
 'living_unit2_BackRow_TopFloor',
 'living_unit3_BackRow_TopFloor']

In [9]:
#review target file object types
target_obj=[]
for obj_type in targetidf.idfobjects:
    for obj in targetidf.idfobjects[obj_type]:
        target_obj.append(obj_type)

unique_target_list=[]        
[unique_target_list.append(x) for x in target_obj if x not in unique_target_list]
unique_target_list

['VERSION',
 'SIMULATIONCONTROL',
 'BUILDING',
 'SURFACECONVECTIONALGORITHM:INSIDE',
 'TIMESTEP',
 'SITE:LOCATION',
 'SIZINGPERIOD:DESIGNDAY',
 'RUNPERIOD',
 'SITE:WATERMAINSTEMPERATURE',
 'SCHEDULETYPELIMITS',
 'SCHEDULE:DAY:HOURLY',
 'SCHEDULE:WEEK:COMPACT',
 'SCHEDULE:YEAR',
 'SCHEDULE:COMPACT',
 'SCHEDULE:CONSTANT',
 'MATERIAL',
 'MATERIAL:NOMASS',
 'MATERIAL:AIRGAP',
 'WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM',
 'WINDOWMATERIAL:GLAZING',
 'WINDOWMATERIAL:BLIND',
 'CONSTRUCTION',
 'GLOBALGEOMETRYRULES',
 'ZONE',
 'BUILDINGSURFACE:DETAILED',
 'WINDOW',
 'DOOR',
 'WINDOWSHADINGCONTROL',
 'SHADING:OVERHANG',
 'GROUNDHEATTRANSFER:CONTROL',
 'GROUNDHEATTRANSFER:SLAB:MATERIALS',
 'GROUNDHEATTRANSFER:SLAB:MATLPROPS',
 'GROUNDHEATTRANSFER:SLAB:BOUNDCONDS',
 'GROUNDHEATTRANSFER:SLAB:BLDGPROPS',
 'GROUNDHEATTRANSFER:SLAB:INSULATION',
 'GROUNDHEATTRANSFER:SLAB:EQUIVALENTSLAB',
 'GROUNDHEATTRANSFER:BASEMENT:SIMPARAMETERS',
 'GROUNDHEATTRANSFER:BASEMENT:MATLPROPS',
 'GROUNDHEATTRANSFER:BASEMENT:INSUL

In [10]:
#Select WH related object types - This step is manual. Copy object types from the unique_target_list
target_WH_list=['SIZING:PLANT',
                'BRANCH', 'BRANCHLIST', 
                'CONNECTOR:SPLITTER', 'CONNECTOR:MIXER', 'CONNECTORLIST',
                'NODELIST',
                'PIPE:ADIABATIC','PUMP:VARIABLESPEED',
                'WATERHEATER:MIXED','WATERHEATER:SIZING','PLANTLOOP',
                'PLANTEQUIPMENTLIST', 'PLANTEQUIPMENTOPERATION:HEATINGLOAD', 'PLANTEQUIPMENTOPERATIONSCHEMES',
                'SETPOINTMANAGER:SCHEDULED', 'WATERUSE:EQUIPMENT', 'WATERUSE:CONNECTIONS', 'CURVE:QUADRATIC', 'CURVE:CUBIC',
                'CURVE:EXPONENT', 'CURVE:BIQUADRATIC']

In [11]:
#review source file objects
source_obj=[]
for obj_type in sourceidf.idfobjects:
    for obj in sourceidf.idfobjects[obj_type]:
        source_obj.append(obj_type)

unique_source_list=[]        
[unique_source_list.append(x) for x in source_obj if x not in unique_source_list]
unique_source_list

['VERSION',
 'SIMULATIONCONTROL',
 'BUILDING',
 'SURFACECONVECTIONALGORITHM:INSIDE',
 'TIMESTEP',
 'SITE:LOCATION',
 'SIZINGPERIOD:DESIGNDAY',
 'RUNPERIOD',
 'SITE:WATERMAINSTEMPERATURE',
 'SCHEDULETYPELIMITS',
 'SCHEDULE:DAY:HOURLY',
 'SCHEDULE:WEEK:COMPACT',
 'SCHEDULE:YEAR',
 'SCHEDULE:COMPACT',
 'SCHEDULE:CONSTANT',
 'MATERIAL',
 'MATERIAL:NOMASS',
 'MATERIAL:AIRGAP',
 'WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM',
 'WINDOWMATERIAL:GLAZING',
 'WINDOWMATERIAL:BLIND',
 'CONSTRUCTION',
 'GLOBALGEOMETRYRULES',
 'ZONE',
 'BUILDINGSURFACE:DETAILED',
 'WINDOW',
 'DOOR',
 'DOOR:INTERZONE',
 'WINDOWSHADINGCONTROL',
 'INTERNALMASS',
 'SHADING:OVERHANG',
 'GROUNDHEATTRANSFER:CONTROL',
 'GROUNDHEATTRANSFER:SLAB:MATERIALS',
 'GROUNDHEATTRANSFER:SLAB:MATLPROPS',
 'GROUNDHEATTRANSFER:SLAB:BOUNDCONDS',
 'GROUNDHEATTRANSFER:SLAB:BLDGPROPS',
 'GROUNDHEATTRANSFER:SLAB:INSULATION',
 'GROUNDHEATTRANSFER:SLAB:EQUIVALENTSLAB',
 'GROUNDHEATTRANSFER:BASEMENT:SIMPARAMETERS',
 'GROUNDHEATTRANSFER:BASEMENT:MATLPROPS',

In [12]:
#Select WH related object types - This step is manual
source_WH_list=['SIZING:PLANT', 
                'COIL:WATERHEATING:AIRTOWATERHEATPUMP:WRAPPED',
                'FAN:SYSTEMMODEL',
                'PIPE:ADIABATIC', 'PIPE:INDOOR',
                'PUMP:CONSTANTSPEED',
                'WATERHEATER:STRATIFIED', 'WATERHEATER:SIZING', 'WATERHEATER:HEATPUMP:WRAPPEDCONDENSER',
                'BRANCH', 'BRANCHLIST', 
                'CONNECTOR:SPLITTER', 'CONNECTOR:MIXER', 'CONNECTORLIST', 'NODELIST',
                'SETPOINTMANAGER:SCHEDULED',
                'PLANTLOOP','PLANTEQUIPMENTLIST','PLANTEQUIPMENTOPERATION:HEATINGLOAD','PLANTEQUIPMENTOPERATIONSCHEMES',
                'WATERUSE:EQUIPMENT', 'WATERUSE:CONNECTIONS',
                'CURVE:QUADRATIC', 'CURVE:CUBIC', 'CURVE:BIQUADRATIC']

In [13]:
#This is the list of WH realted object types that exist in the target file but not the source file. These objects are no longer needed.
WH_dif_list_target=[obj for obj in target_WH_list if obj not in source_WH_list]
WH_dif_list_target

['PUMP:VARIABLESPEED', 'WATERHEATER:MIXED', 'CURVE:EXPONENT']

In [14]:
def keep_objtype_without_keyword(lst, keywords):
    return [item for item in lst if not any(keyword in item for keyword in keywords)]

In [15]:
#Create an exception list, this is user defined. Make sure the keyword(s) is/are capitalized. 
keep_keys=['CURVE']

delete_lst=keep_objtype_without_keyword(WH_dif_list_target, keep_keys)
delete_lst

['PUMP:VARIABLESPEED', 'WATERHEATER:MIXED']

In [16]:
#Clear these unnecessary objects in the target file
for obj in delete_lst:
    targetidf.idfobjects[obj].clear()

# Edit Coil

In [17]:
#Identify WH related COIL object types exist in both files. 
#COIL needs to be capitalized as object types are capitalized in idf files.
COIL_target=[s for s in target_WH_list if "COIL" in s]
COIL_source=[s for s in source_WH_list if "COIL" in s]

print(COIL_target, COIL_source)

[] ['COIL:WATERHEATING:AIRTOWATERHEATPUMP:WRAPPED']


Reme

In [18]:
lst_coil = targetidf.idfobjects['Coil:WaterHeating:AirToWaterHeatPump:Wrapped']
lst_coil

[]

In [19]:
sourceidf.idfobjects['Coil:WaterHeating:AirToWaterHeatPump:Wrapped']

[
Coil:WaterHeating:AirToWaterHeatPump:Wrapped,
    HPWHPlantDXCoil,          !- Name
    2349.6,                   !- Rated Heating Capacity
    2.4,                      !- Rated COP
    0.981,                    !- Rated Sensible Heat Ratio
    19.72,                    !- Rated Evaporator Inlet Air DryBulb Temperature
    13.5,                     !- Rated Evaporator Inlet Air WetBulb Temperature
    48.89,                    !- Rated Condenser Water Temperature
    0.189,                    !- Rated Evaporator Air Flow Rate
    Yes,                      !- Evaporator Fan Power Included in Rated COP
    HPPlantAirInletNode,      !- Evaporator Air Inlet Node Name
    HPPlantAirOutletNode,     !- Evaporator Air Outlet Node Name
    0,                        !- Crankcase Heater Capacity
    10,                       !- Maximum Ambient Temperature for Crankcase Heater Operation
    WetBulbTemperature,       !- Evaporator Air Temperature Type for Curve Objects
    HPWH-Htg-Cap-fT,      

In [20]:
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['Coil:WaterHeating:AirToWaterHeatPump:Wrapped'][-1]
    targetidf.copyidfobject(newclg)
    lst_coil[-1].Name=newclg.Name+' '+zones[i]
    lst_coil[-1].Evaporator_Air_Inlet_Node_Name=newclg.Evaporator_Air_Inlet_Node_Name+' '+zones[i]
    lst_coil[-1].Evaporator_Air_Outlet_Node_Name=newclg.Evaporator_Air_Outlet_Node_Name+' '+zones[i]
        
lst_coil    

[
Coil:WaterHeating:AirToWaterHeatPump:Wrapped,
    HPWHPlantDXCoil living_unit1_FrontRow_BottomFloor,    !- Name
    2349.6,                   !- Rated Heating Capacity
    2.4,                      !- Rated COP
    0.981,                    !- Rated Sensible Heat Ratio
    19.72,                    !- Rated Evaporator Inlet Air DryBulb Temperature
    13.5,                     !- Rated Evaporator Inlet Air WetBulb Temperature
    48.89,                    !- Rated Condenser Water Temperature
    0.189,                    !- Rated Evaporator Air Flow Rate
    Yes,                      !- Evaporator Fan Power Included in Rated COP
    HPPlantAirInletNode living_unit1_FrontRow_BottomFloor,    !- Evaporator Air Inlet Node Name
    HPPlantAirOutletNode living_unit1_FrontRow_BottomFloor,    !- Evaporator Air Outlet Node Name
    0,                        !- Crankcase Heater Capacity
    10,                       !- Maximum Ambient Temperature for Crankcase Heater Operation
    WetBulbTempe

# Edit Pump

In [21]:
#Identify WH related PUMP object types exist in both files. 
#PUMP needs to be capitalized as object types are capitalized in idf files.
PUMP_target=[s for s in target_WH_list if "PUMP:" in s]
PUMP_source=[s for s in source_WH_list if "PUMP:" in s]

print(PUMP_target, PUMP_source)

['PUMP:VARIABLESPEED'] ['COIL:WATERHEATING:AIRTOWATERHEATPUMP:WRAPPED', 'PUMP:CONSTANTSPEED', 'WATERHEATER:HEATPUMP:WRAPPEDCONDENSER']


Here we can ignore the non-pump object types. Since 'Pump:VariableSpeed' only exists in the target file, it's already cleared, therefore we only need to add 'Pump:ConstantSpeed' from source to target file. If a variable-speed pump is preferred, it can be either addressed after the whole electrification modeling, or addressed within the souce file - then we need to change the following object type in the code.

In [22]:
lst_pump = targetidf.idfobjects['Pump:ConstantSpeed']
lst_pump

[]

In [23]:
sourceidf.idfobjects['Pump:ConstantSpeed']

[
Pump:ConstantSpeed,
    SWHSys1 Pump,             !- Name
    SWHSys1 Supply Inlet Node,    !- Inlet Node Name
    SWHSys1 Pump-SWHSys1 Water HeaterNodeviaConnector,    !- Outlet Node Name
    AUTOSIZE,                 !- Design Flow Rate
    0.001,                    !- Design Pump Head
    AUTOSIZE,                 !- Design Power Consumption
    1,                        !- Motor Efficiency
    0,                        !- Fraction of Motor Inefficiencies to Fluid Stream
    Intermittent;             !- Pump Control Type
]

In [24]:
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['Pump:ConstantSpeed'][-1]
    targetidf.copyidfobject(newclg)
    lst_pump[-1].Name=newclg.Name+' '+zones[i]
    lst_pump[-1].Inlet_Node_Name=newclg.Inlet_Node_Name+' '+zones[i]
    lst_pump[-1].Outlet_Node_Name=newclg.Outlet_Node_Name+' '+zones[i]
        
lst_pump 

[
Pump:ConstantSpeed,
    SWHSys1 Pump living_unit1_FrontRow_BottomFloor,    !- Name
    SWHSys1 Supply Inlet Node living_unit1_FrontRow_BottomFloor,    !- Inlet Node Name
    SWHSys1 Pump-SWHSys1 Water HeaterNodeviaConnector living_unit1_FrontRow_BottomFloor,    !- Outlet Node Name
    AUTOSIZE,                 !- Design Flow Rate
    0.001,                    !- Design Pump Head
    AUTOSIZE,                 !- Design Power Consumption
    1,                        !- Motor Efficiency
    0,                        !- Fraction of Motor Inefficiencies to Fluid Stream
    Intermittent;             !- Pump Control Type
, 
Pump:ConstantSpeed,
    SWHSys1 Pump living_unit2_FrontRow_BottomFloor,    !- Name
    SWHSys1 Supply Inlet Node living_unit2_FrontRow_BottomFloor,    !- Inlet Node Name
    SWHSys1 Pump-SWHSys1 Water HeaterNodeviaConnector living_unit2_FrontRow_BottomFloor,    !- Outlet Node Name
    AUTOSIZE,                 !- Design Flow Rate
    0.001,                    !- Design 

# Edit Fan

In [25]:
#Identify WH related FAN object types exist in both files. 
#FAN needs to be capitalized as object types are capitalized in idf files.
FAN_target=[s for s in target_WH_list if "FAN:" in s]
FAN_source=[s for s in source_WH_list if "FAN:" in s]

print(FAN_target, FAN_source)

[] ['FAN:SYSTEMMODEL']


In [26]:
lst_fan=targetidf.idfobjects['Fan:SystemModel']
lst_fan

[]

In [27]:
sourceidf.idfobjects['Fan:SystemModel']

[
Fan:SystemModel,
    HPWHPlantFan,             !- Name
    always_avail,             !- Availability Schedule Name
    HPPlantAirOutletNode,     !- Air Inlet Node Name
    HPPlantFanAirOutletNode,    !- Air Outlet Node Name
    0.2279,                   !- Design Maximum Air Flow Rate
    Discrete,                 !- Speed Control Method
    0,                        !- Electric Power Minimum Flow Rate Fraction
    65,                       !- Design Pressure Rise
    1,                        !- Motor Efficiency
    0,                        !- Motor In Air Stream Fraction
    AUTOSIZE,                 !- Design Electric Power Consumption
    TotalEfficiencyAndPressure,    !- Design Power Sizing Method
    ,                         !- Electric Power Per Unit Flow Rate
    ,                         !- Electric Power Per Unit Flow Rate Per Unit Pressure
    0.1722;                   !- Fan Total Efficiency
]

In [28]:
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['Fan:SystemModel'][-1]
    targetidf.copyidfobject(newclg)
    lst_fan[-1].Name=newclg.Name+' '+zones[i]
    lst_fan[-1].Air_Inlet_Node_Name=newclg.Air_Inlet_Node_Name+' '+zones[i]
    lst_fan[-1].Air_Outlet_Node_Name=newclg.Air_Outlet_Node_Name+' '+zones[i]
        
lst_fan 

[
Fan:SystemModel,
    HPWHPlantFan living_unit1_FrontRow_BottomFloor,    !- Name
    always_avail,             !- Availability Schedule Name
    HPPlantAirOutletNode living_unit1_FrontRow_BottomFloor,    !- Air Inlet Node Name
    HPPlantFanAirOutletNode living_unit1_FrontRow_BottomFloor,    !- Air Outlet Node Name
    0.2279,                   !- Design Maximum Air Flow Rate
    Discrete,                 !- Speed Control Method
    0,                        !- Electric Power Minimum Flow Rate Fraction
    65,                       !- Design Pressure Rise
    1,                        !- Motor Efficiency
    0,                        !- Motor In Air Stream Fraction
    AUTOSIZE,                 !- Design Electric Power Consumption
    TotalEfficiencyAndPressure,    !- Design Power Sizing Method
    ,                         !- Electric Power Per Unit Flow Rate
    ,                         !- Electric Power Per Unit Flow Rate Per Unit Pressure
    0.1722;                   !- Fan Tota

# Edit WH

In [29]:
#Identify WH related WATERHEATER object types exist in both files. 
#WATERHEATER needs to be capitalized as object types are capitalized in idf files.
WH_target=[s for s in target_WH_list if "WATERHEATER" in s]
WH_source=[s for s in source_WH_list if "WATERHEATER" in s]

print(WH_target, WH_source)

['WATERHEATER:MIXED', 'WATERHEATER:SIZING'] ['WATERHEATER:STRATIFIED', 'WATERHEATER:SIZING', 'WATERHEATER:HEATPUMP:WRAPPEDCONDENSER']


None of the WaterHeater object types in the target file are in the source file, implying simple copy-paste - target WaterHeater object types have been cleared at the beginning.

In [30]:
lst_WH = targetidf.idfobjects['WaterHeater:Stratified']
lst_WH

[]

In [31]:
sourceidf.idfobjects['WaterHeater:Stratified']

[
WaterHeater:Stratified,
    Water Heater Tank,        !- Name
    Water Heater,             !- EndUse Subcategory
    0.287691,                 !- Tank Volume
    1.594,                    !- Tank Height
    VerticalCylinder,         !- Tank Shape
    ,                         !- Tank Perimeter
    82.2222,                  !- Maximum Temperature Limit
    MasterSlave,              !- Heater Priority Control
    SWHSys1 Water Heater Setpoint Temperature Schedule Name,    !- Heater 1 Setpoint Temperature Schedule Name
    18.5,                     !- Heater 1 Deadband Temperature Difference
    4500,                     !- Heater 1 Capacity
    1.129,                    !- Heater 1 Height
    SWHSys1 Water Heater Setpoint Temperature Schedule Name,    !- Heater 2 Setpoint Temperature Schedule Name
    18.5,                     !- Heater 2 Deadband Temperature Difference
    0,                        !- Heater 2 Capacity
    0.266,                    !- Heater 2 Height
    Electricity,

In [32]:
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['WaterHeater:Stratified'][-1]
    targetidf.copyidfobject(newclg)
    lst_WH[-1].Name=newclg.Name+' '+zones[i]
    lst_WH[-1].Use_Side_Inlet_Node_Name=newclg.Use_Side_Inlet_Node_Name+' '+zones[i]
    lst_WH[-1].Use_Side_Outlet_Node_Name=newclg.Use_Side_Outlet_Node_Name+' '+zones[i]
        
lst_WH 

[
WaterHeater:Stratified,
    Water Heater Tank living_unit1_FrontRow_BottomFloor,    !- Name
    Water Heater,             !- EndUse Subcategory
    0.287691,                 !- Tank Volume
    1.594,                    !- Tank Height
    VerticalCylinder,         !- Tank Shape
    ,                         !- Tank Perimeter
    82.2222,                  !- Maximum Temperature Limit
    MasterSlave,              !- Heater Priority Control
    SWHSys1 Water Heater Setpoint Temperature Schedule Name,    !- Heater 1 Setpoint Temperature Schedule Name
    18.5,                     !- Heater 1 Deadband Temperature Difference
    4500,                     !- Heater 1 Capacity
    1.129,                    !- Heater 1 Height
    SWHSys1 Water Heater Setpoint Temperature Schedule Name,    !- Heater 2 Setpoint Temperature Schedule Name
    18.5,                     !- Heater 2 Deadband Temperature Difference
    0,                        !- Heater 2 Capacity
    0.266,                    !- He

# Edit HPWH

Now we have coil, fan, pump and water heater, we can now put them all together in the heat pump water heater.

In [33]:
lst_HPWH = targetidf.idfobjects['WaterHeater:HeatPump:WrappedCondenser']
lst_HPWH

[]

In [34]:
sourceidf.idfobjects['WaterHeater:HeatPump:WrappedCondenser']

[
WaterHeater:HeatPump:WrappedCondenser,
    PlantHeatPumpWaterHeater,    !- Name
    always_avail,             !- Availability Schedule Name
    PlantHPWHTempSch,         !- Compressor Setpoint Temperature Schedule Name
    3.89,                     !- Dead Band Temperature Difference
    0.0664166667,             !- Condenser Bottom Location
    0.8634166667,             !- Condenser Top Location
    0.2279,                   !- Evaporator Air Flow Rate
    OutdoorAirOnly,           !- Inlet Air Configuration
    ,                         !- Air Inlet Node Name
    ,                         !- Air Outlet Node Name
    HPPlantAirInletNode,      !- Outdoor Air Node Name
    HPPlantFanAirOutletNode,    !- Exhaust Air Node Name
    ,                         !- Inlet Air Temperature Schedule Name
    ,                         !- Inlet Air Humidity Schedule Name
    ,                         !- Inlet Air Zone Name
    WaterHeater:Stratified,    !- Tank Object Type
    Water Heater Tank,   

In [35]:
for i in range(len(zones)):
    newclg = sourceidf.idfobjects['WaterHeater:HeatPump:WrappedCondenser'][-1]
    targetidf.copyidfobject(newclg)
    lst_HPWH[-1].Name=newclg.Name+' '+zones[i]
    
    lst_HPWH[-1].Outdoor_Air_Node_Name=newclg.Outdoor_Air_Node_Name+' '+zones[i]
    lst_HPWH[-1].Exhaust_Air_Node_Name=newclg.Exhaust_Air_Node_Name+' '+zones[i]
    
    lst_HPWH[-1].Tank_Name=lst_WH[i].Name #Connect to the corresponding water heater
    lst_HPWH[-1].Tank_Use_Side_Inlet_Node_Name=lst_WH[i].Use_Side_Inlet_Node_Name #Connect to the inlet node of the water heater
    lst_HPWH[-1].Tank_Use_Side_Outlet_Node_Name=lst_WH[i].Use_Side_Outlet_Node_Name #Connect to the outlet node of the water heater
    
    lst_HPWH[-1].DX_Coil_Name=lst_coil[i].Name #Connect to the corresponding DX coil
    
    lst_HPWH[-1].Fan_Name=lst_fan[i].Name #Connect to the corresponding system model fan

lst_HPWH

[
WaterHeater:HeatPump:WrappedCondenser,
    PlantHeatPumpWaterHeater living_unit1_FrontRow_BottomFloor,    !- Name
    always_avail,             !- Availability Schedule Name
    PlantHPWHTempSch,         !- Compressor Setpoint Temperature Schedule Name
    3.89,                     !- Dead Band Temperature Difference
    0.0664166667,             !- Condenser Bottom Location
    0.8634166667,             !- Condenser Top Location
    0.2279,                   !- Evaporator Air Flow Rate
    OutdoorAirOnly,           !- Inlet Air Configuration
    ,                         !- Air Inlet Node Name
    ,                         !- Air Outlet Node Name
    HPPlantAirInletNode living_unit1_FrontRow_BottomFloor,    !- Outdoor Air Node Name
    HPPlantFanAirOutletNode living_unit1_FrontRow_BottomFloor,    !- Exhaust Air Node Name
    ,                         !- Inlet Air Temperature Schedule Name
    ,                         !- Inlet Air Humidity Schedule Name
    ,                        

## Edit WaterHeater:Sizing 

In [36]:
lst_WHSizing=targetidf.idfobjects['WaterHeater:Sizing']
lst_WHSizing

[
WaterHeater:Sizing,
    Water Heater_unit1_FrontRow_BottomFloor,    !- WaterHeater Name
    ResidentialHUD-FHAMinimum,    !- Design Mode
    ,                         !- Time Storage Can Meet Peak Draw
    ,                         !- Time for Tank Recovery
    ,                         !- Nominal Tank Volume for Autosizing Plant Connections
    2,                        !- Number of Bedrooms
    2;                        !- Number of Bathrooms
, 
WaterHeater:Sizing,
    Water Heater_unit2_FrontRow_BottomFloor,    !- WaterHeater Name
    ResidentialHUD-FHAMinimum,    !- Design Mode
    ,                         !- Time Storage Can Meet Peak Draw
    ,                         !- Time for Tank Recovery
    ,                         !- Nominal Tank Volume for Autosizing Plant Connections
    2,                        !- Number of Bedrooms
    2;                        !- Number of Bathrooms
, 
WaterHeater:Sizing,
    Water Heater_unit3_FrontRow_BottomFloor,    !- WaterHeater Name
    Re

In [37]:
sourceidf.idfobjects['WaterHeater:Sizing']

[
WaterHeater:Sizing,
    Water Heater Tank,        !- WaterHeater Name
    ResidentialHUD-FHAMinimum,    !- Design Mode
    ,                         !- Time Storage Can Meet Peak Draw
    ,                         !- Time for Tank Recovery
    ,                         !- Nominal Tank Volume for Autosizing Plant Connections
    3,                        !- Number of Bedrooms
    3;                        !- Number of Bathrooms
]

To properly size WHs, we need to keep the number of bedrooms and bathrooms in the target file. Therefore, instead of clearing the objects from target file, we simply replace the WaterHeater Name.

In [38]:
for i in range(len(zones)):
    lst_WHSizing[i].WaterHeater_Name=lst_WH[i].Name

lst_WHSizing

[
WaterHeater:Sizing,
    Water Heater Tank living_unit1_FrontRow_BottomFloor,    !- WaterHeater Name
    ResidentialHUD-FHAMinimum,    !- Design Mode
    ,                         !- Time Storage Can Meet Peak Draw
    ,                         !- Time for Tank Recovery
    ,                         !- Nominal Tank Volume for Autosizing Plant Connections
    2,                        !- Number of Bedrooms
    2;                        !- Number of Bathrooms
, 
WaterHeater:Sizing,
    Water Heater Tank living_unit2_FrontRow_BottomFloor,    !- WaterHeater Name
    ResidentialHUD-FHAMinimum,    !- Design Mode
    ,                         !- Time Storage Can Meet Peak Draw
    ,                         !- Time for Tank Recovery
    ,                         !- Nominal Tank Volume for Autosizing Plant Connections
    2,                        !- Number of Bedrooms
    2;                        !- Number of Bathrooms
, 
WaterHeater:Sizing,
    Water Heater Tank living_unit3_FrontRow_Bottom

# Edit PlantEquipment

In [39]:
#Identify WH related PLANTEQUIPMENT object types exist in both files. 
#PLANTEQUIPMENT needs to be capitalized as object types are capitalized in idf files.
PE_target=[s for s in target_WH_list if "PLANTEQUIPMENT" in s]
PE_source=[s for s in source_WH_list if "PLANTEQUIPMENT" in s]

print(PE_target, PE_source)

['PLANTEQUIPMENTLIST', 'PLANTEQUIPMENTOPERATION:HEATINGLOAD', 'PLANTEQUIPMENTOPERATIONSCHEMES'] ['PLANTEQUIPMENTLIST', 'PLANTEQUIPMENTOPERATION:HEATINGLOAD', 'PLANTEQUIPMENTOPERATIONSCHEMES']


Identical object types for both files, so we will replace the objects in each type one by one.

### Plant Equipment List 

With all the HPWH set up, we can now put the heat pump and water heater together in the PlantEquipmentList.

In [40]:
lst_PEqp = targetidf.idfobjects['PlantEquipmentList']
lst_PEqp

[
PlantEquipmentList,
    DHW Plant Equipment_unit1_FrontRow_BottomFloor,    !- Name
    WaterHeater:Mixed,        !- Equipment 1 Object Type
    Water Heater_unit1_FrontRow_BottomFloor;    !- Equipment 1 Name
, 
PlantEquipmentList,
    DHW Plant Equipment_unit2_FrontRow_BottomFloor,    !- Name
    WaterHeater:Mixed,        !- Equipment 1 Object Type
    Water Heater_unit2_FrontRow_BottomFloor;    !- Equipment 1 Name
, 
PlantEquipmentList,
    DHW Plant Equipment_unit3_FrontRow_BottomFloor,    !- Name
    WaterHeater:Mixed,        !- Equipment 1 Object Type
    Water Heater_unit3_FrontRow_BottomFloor;    !- Equipment 1 Name
, 
PlantEquipmentList,
    DHW Plant Equipment_unit1_BackRow_BottomFloor,    !- Name
    WaterHeater:Mixed,        !- Equipment 1 Object Type
    Water Heater_unit1_BackRow_BottomFloor;    !- Equipment 1 Name
, 
PlantEquipmentList,
    DHW Plant Equipment_unit2_BackRow_BottomFloor,    !- Name
    WaterHeater:Mixed,        !- Equipment 1 Object Type
    Water Heater_

In [41]:
sourceidf.idfobjects['PlantEquipmentList']

[
PlantEquipmentList,
    SWHSys1 Equipment List,    !- Name
    WaterHeater:HeatPump:WrappedCondenser,    !- Equipment 1 Object Type
    PlantHeatPumpWaterHeater;    !- Equipment 1 Name
]

Since the original PlantEquipmentList uses a different type of water heater, we need to clear the existing plant equipment list then add the new objects.

In [42]:
targetidf.idfobjects['PlantEquipmentList'].clear()

for i in range(len(zones)):
    new=sourceidf.idfobjects['PlantEquipmentList'][0]
    targetidf.copyidfobject(new)
    lst_PEqp[-1].Name=new.Name+' '+zones[i]
    lst_PEqp[-1].Equipment_1_Name=lst_HPWH[i].Name #Connect to the corresponding HPWH

lst_PEqp

[
PlantEquipmentList,
    SWHSys1 Equipment List living_unit1_FrontRow_BottomFloor,    !- Name
    WaterHeater:HeatPump:WrappedCondenser,    !- Equipment 1 Object Type
    PlantHeatPumpWaterHeater living_unit1_FrontRow_BottomFloor;    !- Equipment 1 Name
, 
PlantEquipmentList,
    SWHSys1 Equipment List living_unit2_FrontRow_BottomFloor,    !- Name
    WaterHeater:HeatPump:WrappedCondenser,    !- Equipment 1 Object Type
    PlantHeatPumpWaterHeater living_unit2_FrontRow_BottomFloor;    !- Equipment 1 Name
, 
PlantEquipmentList,
    SWHSys1 Equipment List living_unit3_FrontRow_BottomFloor,    !- Name
    WaterHeater:HeatPump:WrappedCondenser,    !- Equipment 1 Object Type
    PlantHeatPumpWaterHeater living_unit3_FrontRow_BottomFloor;    !- Equipment 1 Name
, 
PlantEquipmentList,
    SWHSys1 Equipment List living_unit1_BackRow_BottomFloor,    !- Name
    WaterHeater:HeatPump:WrappedCondenser,    !- Equipment 1 Object Type
    PlantHeatPumpWaterHeater living_unit1_BackRow_BottomFloor;   

### Plant Equipment Operation

In [43]:
lst_POEqpHL=targetidf.idfobjects['PlantEquipmentOperation:HeatingLoad']
lst_POEqpHL

[
PlantEquipmentOperation:HeatingLoad,
    DHW Control Scheme_unit1_FrontRow_BottomFloor,    !- Name
    0,                        !- Load Range 1 Lower Limit
    1000000000000000,         !- Load Range 1 Upper Limit
    DHW Plant Equipment_unit1_FrontRow_BottomFloor;    !- Range 1 Equipment List Name
, 
PlantEquipmentOperation:HeatingLoad,
    DHW Control Scheme_unit2_FrontRow_BottomFloor,    !- Name
    0,                        !- Load Range 1 Lower Limit
    1000000000000000,         !- Load Range 1 Upper Limit
    DHW Plant Equipment_unit2_FrontRow_BottomFloor;    !- Range 1 Equipment List Name
, 
PlantEquipmentOperation:HeatingLoad,
    DHW Control Scheme_unit3_FrontRow_BottomFloor,    !- Name
    0,                        !- Load Range 1 Lower Limit
    1000000000000000,         !- Load Range 1 Upper Limit
    DHW Plant Equipment_unit3_FrontRow_BottomFloor;    !- Range 1 Equipment List Name
, 
PlantEquipmentOperation:HeatingLoad,
    DHW Control Scheme_unit1_BackRow_BottomFloor,

In [44]:
sourceidf.idfobjects['PlantEquipmentOperation:HeatingLoad']

[
PlantEquipmentOperation:HeatingLoad,
    SWHSys1 Operation Scheme,    !- Name
    0,                        !- Load Range 1 Lower Limit
    1000000000000000,         !- Load Range 1 Upper Limit
    SWHSys1 Equipment List;    !- Range 1 Equipment List Name
]

In [45]:
targetidf.idfobjects['PlantEquipmentOperation:HeatingLoad'].clear()

for i in range(len(zones)):
    new=sourceidf.idfobjects['PlantEquipmentOperation:HeatingLoad'][0]
    targetidf.copyidfobject(new)
    lst_POEqpHL[-1].Name=new.Name+' '+zones[i]
    lst_POEqpHL[-1].Range_1_Equipment_List_Name=lst_PEqp[i].Name # Connect to the corresponding Plant Equipment List

targetidf.idfobjects['PlantEquipmentOperation:HeatingLoad']

[
PlantEquipmentOperation:HeatingLoad,
    SWHSys1 Operation Scheme living_unit1_FrontRow_BottomFloor,    !- Name
    0,                        !- Load Range 1 Lower Limit
    1000000000000000,         !- Load Range 1 Upper Limit
    SWHSys1 Equipment List living_unit1_FrontRow_BottomFloor;    !- Range 1 Equipment List Name
, 
PlantEquipmentOperation:HeatingLoad,
    SWHSys1 Operation Scheme living_unit2_FrontRow_BottomFloor,    !- Name
    0,                        !- Load Range 1 Lower Limit
    1000000000000000,         !- Load Range 1 Upper Limit
    SWHSys1 Equipment List living_unit2_FrontRow_BottomFloor;    !- Range 1 Equipment List Name
, 
PlantEquipmentOperation:HeatingLoad,
    SWHSys1 Operation Scheme living_unit3_FrontRow_BottomFloor,    !- Name
    0,                        !- Load Range 1 Lower Limit
    1000000000000000,         !- Load Range 1 Upper Limit
    SWHSys1 Equipment List living_unit3_FrontRow_BottomFloor;    !- Range 1 Equipment List Name
, 
PlantEquipmentOpe

### Plant Equipment Operation Schemes

In [46]:
lst_POEqpSch=targetidf.idfobjects['PlantEquipmentOperationSchemes']
lst_POEqpSch

[
PlantEquipmentOperationSchemes,
    DHW Loop Operation_unit1_FrontRow_BottomFloor,    !- Name
    PlantEquipmentOperation:HeatingLoad,    !- Control Scheme 1 Object Type
    DHW Control Scheme_unit1_FrontRow_BottomFloor,    !- Control Scheme 1 Name
    always_avail;             !- Control Scheme 1 Schedule Name
, 
PlantEquipmentOperationSchemes,
    DHW Loop Operation_unit2_FrontRow_BottomFloor,    !- Name
    PlantEquipmentOperation:HeatingLoad,    !- Control Scheme 1 Object Type
    DHW Control Scheme_unit2_FrontRow_BottomFloor,    !- Control Scheme 1 Name
    always_avail;             !- Control Scheme 1 Schedule Name
, 
PlantEquipmentOperationSchemes,
    DHW Loop Operation_unit3_FrontRow_BottomFloor,    !- Name
    PlantEquipmentOperation:HeatingLoad,    !- Control Scheme 1 Object Type
    DHW Control Scheme_unit3_FrontRow_BottomFloor,    !- Control Scheme 1 Name
    always_avail;             !- Control Scheme 1 Schedule Name
, 
PlantEquipmentOperationSchemes,
    DHW Loop Opera

In [47]:
sourceidf.idfobjects['PlantEquipmentOperationSchemes']

[
PlantEquipmentOperationSchemes,
    SWHSys1 Loop Operation Scheme List,    !- Name
    PlantEquipmentOperation:HeatingLoad,    !- Control Scheme 1 Object Type
    SWHSys1 Operation Scheme,    !- Control Scheme 1 Name
    always_avail;             !- Control Scheme 1 Schedule Name
]

In [48]:
targetidf.idfobjects['PlantEquipmentOperationSchemes'].clear()

for i in range(len(zones)):
    new=sourceidf.idfobjects['PlantEquipmentOperationSchemes'][0]
    targetidf.copyidfobject(new)
    lst_POEqpSch[-1].Name=new.Name+' '+zones[i]
    lst_POEqpSch[-1].Control_Scheme_1_Name=lst_POEqpHL[i].Name #Connect to the corresponding Plant Equipment Operation Heating Load
    
lst_POEqpSch

[
PlantEquipmentOperationSchemes,
    SWHSys1 Loop Operation Scheme List living_unit1_FrontRow_BottomFloor,    !- Name
    PlantEquipmentOperation:HeatingLoad,    !- Control Scheme 1 Object Type
    SWHSys1 Operation Scheme living_unit1_FrontRow_BottomFloor,    !- Control Scheme 1 Name
    always_avail;             !- Control Scheme 1 Schedule Name
, 
PlantEquipmentOperationSchemes,
    SWHSys1 Loop Operation Scheme List living_unit2_FrontRow_BottomFloor,    !- Name
    PlantEquipmentOperation:HeatingLoad,    !- Control Scheme 1 Object Type
    SWHSys1 Operation Scheme living_unit2_FrontRow_BottomFloor,    !- Control Scheme 1 Name
    always_avail;             !- Control Scheme 1 Schedule Name
, 
PlantEquipmentOperationSchemes,
    SWHSys1 Loop Operation Scheme List living_unit3_FrontRow_BottomFloor,    !- Name
    PlantEquipmentOperation:HeatingLoad,    !- Control Scheme 1 Object Type
    SWHSys1 Operation Scheme living_unit3_FrontRow_BottomFloor,    !- Control Scheme 1 Name
    alway

# Edit Pipe

Although Pipe is part of the plant system, without touching the configurations of water use equipment, pipes are better to be left untouched, as they are not affected by the water heater replacement.

# Edit Branch

Unlike electrifying air-side system, when electrifying water-side system, we prefer branches to be touched at a minimum level as there are no significant configuration changes. The impact of electrifying water heater on branches is the change of component type and the associated nodes within the existing configurations.

In [98]:
def find_Water_branch(keywords, idf, obj_type):
    lst_objtype=idf.idfobjects[obj_type] #Pull out all branches into a list for an idf
    lst_objtype_name=[]
    
    #check if any of the keywords exist in the branch name
    for i in range(len(lst_objtype)):
        component_type=lst_objtype[i].Component_1_Object_Type
        if any(keyword in component_type for keyword in keywords):
            lst_objtype_name.append(True)
        else:
            lst_objtype_name.append(False)
    
    #Get a list of indices of AC branches in the idf
    lst_Water_objtype_idx=[i for i,x in enumerate(lst_objtype_name) if x] 
    return lst_Water_objtype_idx

In [99]:
lst_branch=targetidf.idfobjects['Branch']

1. Swithc existing WH branches with the new HPWH.

In [56]:
key_WH=['WaterHeater']
lst_target_WH_branch_idx=find_Water_branch(key_WH,targetidf,'Branch')
lst_target_WH_branch_idx

[27,
 37,
 47,
 57,
 67,
 77,
 87,
 97,
 107,
 117,
 127,
 137,
 147,
 157,
 167,
 177,
 187,
 197]

In [57]:
len(lst_target_WH_branch_idx)==len(lst_HPWH)

True

In [58]:
for i in range(len(lst_HPWH)):
    WH_branch_idx=lst_target_WH_branch_idx[i]
    lst_branch[WH_branch_idx].Component_1_Object_Type='WaterHeater:HeatPump:WrappedCondenser'
    lst_branch[WH_branch_idx].Component_1_Name=lst_HPWH[i].Name
    lst_branch[WH_branch_idx].Component_1_Inlet_Node_Name=lst_HPWH[i].Tank_Use_Side_Inlet_Node_Name
    lst_branch[WH_branch_idx].Component_1_Outlet_Node_Name=lst_HPWH[i].Tank_Use_Side_Outlet_Node_Name


lst_branch

[
Branch,
    Air Loop Main Branch_unit1_FrontRow_BottomFloor,    !- Name
    ,                         !- Pressure Drop Curve Name
    AirLoopHVAC:UnitaryHeatCool,    !- Component 1 Object Type
    ACandF_unit1_FrontRow_BottomFloor,    !- Component 1 Name
    Air Loop Inlet Node_unit1_FrontRow_BottomFloor,    !- Component 1 Inlet Node Name
    Air loop outlet node_unit1_FrontRow_BottomFloor;    !- Component 1 Outlet Node Name
, 
Branch,
    Air Loop Main Branch_unit2_FrontRow_BottomFloor,    !- Name
    ,                         !- Pressure Drop Curve Name
    AirLoopHVAC:UnitaryHeatCool,    !- Component 1 Object Type
    ACandF_unit2_FrontRow_BottomFloor,    !- Component 1 Name
    Air Loop Inlet Node_unit2_FrontRow_BottomFloor,    !- Component 1 Inlet Node Name
    Air loop outlet node_unit2_FrontRow_BottomFloor;    !- Component 1 Outlet Node Name
, 
Branch,
    Air Loop Main Branch_unit3_FrontRow_BottomFloor,    !- Name
    ,                         !- Pressure Drop Curve Name
    

2. Switch existing pump branches with new pumps

In [59]:
key_pump=['Pump:VariableSpeed']
lst_target_pump_branch_idx=find_Water_branch(key_pump,targetidf,'Branch')
lst_target_pump_branch_idx

[18,
 28,
 38,
 48,
 58,
 68,
 78,
 88,
 98,
 108,
 118,
 128,
 138,
 148,
 158,
 168,
 178,
 188]

In [60]:
len(lst_target_pump_branch_idx)==len(lst_pump)

True

In [61]:
for i in range(len(lst_pump)):
    pump_branch_idx=lst_target_pump_branch_idx[i]
    lst_branch[pump_branch_idx].Component_1_Object_Type='Pump:ConstantSpeed'
    lst_branch[pump_branch_idx].Component_1_Name=lst_pump[i].Name
    lst_branch[pump_branch_idx].Component_1_Inlet_Node_Name=lst_pump[i].Inlet_Node_Name
    lst_branch[pump_branch_idx].Component_1_Outlet_Node_Name=lst_pump[i].Outlet_Node_Name

lst_branch

[
Branch,
    Air Loop Main Branch_unit1_FrontRow_BottomFloor,    !- Name
    ,                         !- Pressure Drop Curve Name
    AirLoopHVAC:UnitaryHeatCool,    !- Component 1 Object Type
    ACandF_unit1_FrontRow_BottomFloor,    !- Component 1 Name
    Air Loop Inlet Node_unit1_FrontRow_BottomFloor,    !- Component 1 Inlet Node Name
    Air loop outlet node_unit1_FrontRow_BottomFloor;    !- Component 1 Outlet Node Name
, 
Branch,
    Air Loop Main Branch_unit2_FrontRow_BottomFloor,    !- Name
    ,                         !- Pressure Drop Curve Name
    AirLoopHVAC:UnitaryHeatCool,    !- Component 1 Object Type
    ACandF_unit2_FrontRow_BottomFloor,    !- Component 1 Name
    Air Loop Inlet Node_unit2_FrontRow_BottomFloor,    !- Component 1 Inlet Node Name
    Air loop outlet node_unit2_FrontRow_BottomFloor;    !- Component 1 Outlet Node Name
, 
Branch,
    Air Loop Main Branch_unit3_FrontRow_BottomFloor,    !- Name
    ,                         !- Pressure Drop Curve Name
    

# BranchList, Connector, Splitter and Mixer

Again, since there are no major configuration changes in the water system, these object types can be left untouched.

# Edit Plant Lopp

In [76]:
#Identify WH related PLANTLOOP object types exist in both files. 
#PLANTLOOP needs to be capitalized as object types are capitalized in idf files.
PLANTLOOP_target=[s for s in target_WH_list if "PLANTLOOP" in s]
PLANTLOOP_source=[s for s in source_WH_list if "PLANTLOOP" in s]

print(PLANTLOOP_target, PLANTLOOP_source)

['PLANTLOOP'] ['PLANTLOOP']


In [77]:
lst_PL = targetidf.idfobjects['PlantLoop']
lst_PL

[
PlantLoop,
    DHW Loop_unit1_FrontRow_BottomFloor,    !- Name
    Water,                    !- Fluid Type
    ,                         !- User Defined Fluid Type
    DHW Loop Operation_unit1_FrontRow_BottomFloor,    !- Plant Equipment Operation Scheme Name
    DHW Supply Outlet Node_unit1_FrontRow_BottomFloor,    !- Loop Temperature Setpoint Node Name
    100,                      !- Maximum Loop Temperature
    0,                        !- Minimum Loop Temperature
    autosize,                 !- Maximum Loop Flow Rate
    0,                        !- Minimum Loop Flow Rate
    autocalculate,            !- Plant Loop Volume
    Mains Inlet Node_unit1_FrontRow_BottomFloor,    !- Plant Side Inlet Node Name
    DHW Supply Outlet Node_unit1_FrontRow_BottomFloor,    !- Plant Side Outlet Node Name
    DHW Supply Branches_unit1_FrontRow_BottomFloor,    !- Plant Side Branch List Name
    DHW Supply Connectors_unit1_FrontRow_BottomFloor,    !- Plant Side Connector List Name
    DHW Demand 

For PlantLoop objects, we need to update the name of plant equipment operation scheme and the plant side inlet node name, which are affected by the new pumps and the new water heaters. Other things can remain untouched.

In [78]:
for i in range(len(lst_PL)):
    lst_PL[i].Plant_Equipment_Operation_Scheme_Name=lst_POEqpSch[i].Name
    lst_PL[i].Plant_Side_Inlet_Node_Name=lst_pump[i].Inlet_Node_Name

lst_PL

[
PlantLoop,
    DHW Loop_unit1_FrontRow_BottomFloor,    !- Name
    Water,                    !- Fluid Type
    ,                         !- User Defined Fluid Type
    SWHSys1 Loop Operation Scheme List living_unit1_FrontRow_BottomFloor,    !- Plant Equipment Operation Scheme Name
    DHW Supply Outlet Node_unit1_FrontRow_BottomFloor,    !- Loop Temperature Setpoint Node Name
    100,                      !- Maximum Loop Temperature
    0,                        !- Minimum Loop Temperature
    autosize,                 !- Maximum Loop Flow Rate
    0,                        !- Minimum Loop Flow Rate
    autocalculate,            !- Plant Loop Volume
    SWHSys1 Supply Inlet Node living_unit1_FrontRow_BottomFloor,    !- Plant Side Inlet Node Name
    DHW Supply Outlet Node_unit1_FrontRow_BottomFloor,    !- Plant Side Outlet Node Name
    DHW Supply Branches_unit1_FrontRow_BottomFloor,    !- Plant Side Branch List Name
    DHW Supply Connectors_unit1_FrontRow_BottomFloor,    !- Plant S

For reference, following fieldnames from different objects should be aligned:

1. PlantLoop.Plant_Side_Inlet_Node_Name = 
   Pump.inlet_node_name = 
   Branch(Supply Inlet Branch).Component_1_Inlet_Node_Name

2. PlantLoop.Plant_Side_Outlet_Node_Name=PlantLoop.Loop_Temperature_Setpoint_Node_Name=SetpointManager.Setpoint_Node_or_NodeList_Name=Pipe(supply outlet pipe).Outlet_Node_Name=Branch(Supply Outlet Brach).Component_1_Outlet_Node_Name


3. PlantLoop.Plant_Side_Branch_List_Name=BranchList(Supply Branches).Name

4. PlantLoop.Demand_Side_Inlet_Node_Name=Pipe(Demand Inlet Pipe).Inlet_Node_Name=Branch(Demand Inlet Branch).Component_1_Inlet_Node_Name

5. PlantLoop.Demand_Side_Outlet_Node_Name=Branch(Mains Makeup Branch).Component_1_Outlet_Node_Name

# Edit Sizing:Plant

Since we didn't change the plant loop names, Sizing:Plant object should also remain untouched

# Curves

Curves and Schedules are different from the previous object types. We want to make sure all the new curves and schedules are migrated properly to avoid any errors, but also want to avoid duplicated names in Curves and Schedules. For curves, the safest way to do this is to duplicate all the Curves in the source file and then check any duplicated names exist. We will then drop the object with the duplicated name in the TARGET file - the first object that show up. Below are the functions to do so.

In [82]:
def find_dup_and_identify_index(target_lst):
    
    # Dictionary to track the indices of duplicates
    index_dict = {}
    duplicates = []
    
    # Identify indices of duplicated items
    for idx, item in enumerate(target_lst):
        if item in index_dict:
            duplicates.append(index_dict[item])  # Record the first occurrence index
            index_dict[item] = idx  # Update to the latest index
        else:
            index_dict[item] = idx
    
    return duplicates

In [83]:
def drop_target_duplication(index_lst, idf, obj_type):
    
    #If the length of duplication index list is not zero, pop the curve objet, else do nothing
    if len(index_lst) > 0:
        
        #VERY IMPORTANT: Iterate backwards over the indices to avoid shifting issues
        for idx in reversed(index_lst):
            idf.idfobjects[obj_type].pop(idx)
        

In [84]:
#Identify HVAC related CURVE object types exist in both files. 
#AVAILABILITY needs to be capitalized as object types are capitalized in idf files.
curve_target=[s for s in target_WH_list if "CURVE" in s]
curve_source=[s for s in source_WH_list if "CURVE" in s]

#AirLoopHVAC can be different object types in target and source files. Print out the object types in sequence of target and source.
print(curve_target, curve_source)

['CURVE:QUADRATIC', 'CURVE:CUBIC', 'CURVE:EXPONENT', 'CURVE:BIQUADRATIC'] ['CURVE:QUADRATIC', 'CURVE:CUBIC', 'CURVE:BIQUADRATIC']


This means that no new type of curve is involved in the source file. This can make the curve replacement and addition process much easier. We can handle all types of Curve all together.

In [85]:
if curve_target == curve_source:
    
    for curve_type in curve_source:
        lst_curve = targetidf.idfobjects[curve_type]
        new_curves = sourceidf.idfobjects[curve_type][:] #We will migrate all curves from the source file
        
        for i in range(len(new_curves)):
            targetidf.copyidfobject(new_curves[i])
        
        #Now we start the duplication check
        lst_curve_name=[]
        for i in range(len(lst_curve)):
            lst_curve_name.append(lst_curve[i].Name)
        dup_curve_index=find_dup_and_identify_index(lst_curve_name)
        
        #If any duplicated curve names exist, drop the first one that existed in the original target file
        drop_target_duplication(dup_curve_index, targetidf, curve_type)

In case the target and source files have different Curve types, we will simply repeat the above process but adding new curve types from the source file to the target file.

In [86]:
if curve_target != curve_source:
    
    for i in range(len(curve_source)):
        
        #When there are new curve types in the source file
        if curve_source[i] not in curve_target:
            lst_curve = targetidf.idfobjects[curve_source[i]] #Supposed to be an empty list
            new_curves = sourceidf.idfobjects[curve_source[i]][:]
            for a in range(len(new_curves)):
                targetidf.copyidfobject(new_curves[a]) #No need to check duplications
        
        #Then repeat the same process for those same existing curve types
        elif curve_source[i] in curve_target:
            lst_curve = targetidf.idfobjects[curve_source[i]] #Supposed to be a non-empty list
            new_curves = sourceidf.idfobjects[curve_source[i]][:]
            for b in range(len(new_curves)):
                targetidf.copyidfobject(new_curves[b]) #Need to check duplications
            #Check for duplications
            lst_curve_name=[]
            for c in range(len(lst_curve)):
                lst_curve_name.append(lst_curve[c].Name)
            dup_curve_index=find_dup_and_identify_index(lst_curve_name)
            #If any duplicated curve names exist, drop the first one that existed in the original target file
            drop_target_duplication(dup_curve_index, targetidf, curve_source[i])

# Schedule

The remaining item is to check if any shedules should be added from source to target. We first review all used object types with all associated fieldnames in the target file.

In [87]:
#Create two empty lists, one for object type, one for the associated fieldnames
obj_types=[]
obj_type_fieldnames=[]

#Pull out all used object type and associated fieldnames
for obj_type in source_WH_list:
    if len(targetidf.idfobjects[obj_type]) > 0:
        obj=targetidf.idfobjects[obj_type][0]
        obj_types.append(obj_type)
        fieldnames=obj.fieldnames
        obj_type_fieldnames.append(fieldnames)

#Organize the object types and fieldnames into a dictionary
obj_fieldnames_dict=dict(zip(obj_types, obj_type_fieldnames))
obj_fieldnames_dict

{'Sizing:ZONE': ['key',
  'Zone_or_ZoneList_Name',
  'Zone_Cooling_Design_Supply_Air_Temperature_Input_Method',
  'Zone_Cooling_Design_Supply_Air_Temperature',
  'Zone_Cooling_Design_Supply_Air_Temperature_Difference',
  'Zone_Heating_Design_Supply_Air_Temperature_Input_Method',
  'Zone_Heating_Design_Supply_Air_Temperature',
  'Zone_Heating_Design_Supply_Air_Temperature_Difference',
  'Zone_Cooling_Design_Supply_Air_Humidity_Ratio',
  'Zone_Heating_Design_Supply_Air_Humidity_Ratio',
  'Design_Specification_Outdoor_Air_Object_Name',
  'Zone_Heating_Sizing_Factor',
  'Zone_Cooling_Sizing_Factor',
  'Cooling_Design_Air_Flow_Method',
  'Cooling_Design_Air_Flow_Rate',
  'Cooling_Minimum_Air_Flow_per_Zone_Floor_Area',
  'Cooling_Minimum_Air_Flow',
  'Cooling_Minimum_Air_Flow_Fraction',
  'Heating_Design_Air_Flow_Method',
  'Heating_Design_Air_Flow_Rate',
  'Heating_Maximum_Air_Flow_per_Zone_Floor_Area',
  'Heating_Maximum_Air_Flow',
  'Heating_Maximum_Air_Flow_Fraction',
  'Design_Specifica

Now we narrow down the dictionary to those fieldnames containing "Schedule". We only keep the object type and its fieldnames that have "Schedule".

In [88]:
schedule_key='Schedule'
obj_fieldnames_dict_with_schedule = {
    k: [value for value in v if schedule_key in value]
    for k, v in obj_fieldnames_dict.items() 
    if any(schedule_key in value for value in v)
}
obj_fieldnames_dict_with_schedule

{'Sizing:ZONE': ['Zone_Humidistat_Dehumidification_Set_Point_Schedule_Name',
  'Zone_Humidistat_Humidification_Set_Point_Schedule_Name'],
 'FAN:SYSTEMMODEL': ['Availability_Schedule_Name'],
 'PUMP:CONSTANTSPEED': ['Pump_Flow_Rate_Schedule_Name'],
 'WATERHEATER:STRATIFIED': ['Heater_1_Setpoint_Temperature_Schedule_Name',
  'Heater_2_Setpoint_Temperature_Schedule_Name',
  'Ambient_Temperature_Schedule_Name',
  'Use_Flow_Rate_Fraction_Schedule_Name',
  'Cold_Water_Supply_Temperature_Schedule_Name',
  'Indirect_Alternate_Setpoint_Temperature_Schedule_Name'],
 'WATERHEATER:HEATPUMP:WRAPPEDCONDENSER': ['Availability_Schedule_Name',
  'Compressor_Setpoint_Temperature_Schedule_Name',
  'Inlet_Air_Temperature_Schedule_Name',
  'Inlet_Air_Humidity_Schedule_Name',
  'Compressor_Ambient_Temperature_Schedule_Name',
  'Inlet_Air_Mixer_Schedule_Name'],
 'SETPOINTMANAGER:SCHEDULED': ['Schedule_Name'],
 'PLANTEQUIPMENTOPERATIONSCHEMES': ['Control_Scheme_1_Schedule_Name',
  'Control_Scheme_2_Schedule_Na

Dissecting the dictionary, we have dictionary keys indicating object types that use schedule as input.

In [89]:
obj_type_with_schedule=list(obj_fieldnames_dict_with_schedule.keys())
obj_type_with_schedule

['Sizing:ZONE',
 'FAN:SYSTEMMODEL',
 'PUMP:CONSTANTSPEED',
 'WATERHEATER:STRATIFIED',
 'WATERHEATER:HEATPUMP:WRAPPEDCONDENSER',
 'SETPOINTMANAGER:SCHEDULED',
 'PLANTEQUIPMENTOPERATIONSCHEMES',
 'WATERUSE:EQUIPMENT',
 'WATERUSE:CONNECTIONS']

Now we can pull out all schedules used by the new WH system.

In [90]:
used_schedules = []
for obj_type in obj_type_with_schedule:
    
    #Pull out the list of fieldnames with "Schedule"
    schedule_field = obj_fieldnames_dict_with_schedule[obj_type]
    
    #Check each field and pull out the name of the schedule used
    for field in schedule_field:
        schedule = getattr(targetidf.idfobjects[obj_type][0], field, None)
        if schedule != None:
            used_schedules.append(schedule)

#Remove empty schedules
used_schedules = [sch for sch in used_schedules if sch != '']

In [91]:
used_schedules

['always_avail',
 'SWHSys1 Water Heater Setpoint Temperature Schedule Name',
 'SWHSys1 Water Heater Setpoint Temperature Schedule Name',
 'SWHSys1 Water Heater Ambient Temperature Schedule Name',
 'always_avail',
 'PlantHPWHTempSch',
 'DHWSupplySetpoint',
 'always_avail',
 'ClothesWasher',
 'CWWaterTempSchedule']

Now we go back to the existing schedules. There are a few different object types for schedules.

In [92]:
schedule_obj_type = [sch for sch in unique_target_list if 'SCHEDULE' in sch]
schedule_obj_type

['SCHEDULETYPELIMITS',
 'SCHEDULE:DAY:HOURLY',
 'SCHEDULE:WEEK:COMPACT',
 'SCHEDULE:YEAR',
 'SCHEDULE:COMPACT',
 'SCHEDULE:CONSTANT',
 'AVAILABILITYMANAGER:SCHEDULED',
 'SETPOINTMANAGER:SCHEDULED']

We review each schedule type and pull out the name of all existing schedules.

In [93]:
existing_schedules=[]
for sch_type in schedule_obj_type:
    for i in range(len(targetidf.idfobjects[sch_type])):
        sch = targetidf.idfobjects[sch_type][i].Name
        existing_schedules.append(sch)

existing_schedules

['any number',
 'On/Off',
 'control_type',
 'fraction',
 'Temperature',
 'OccupancyDay',
 'LightingDay',
 'LightingDay_EELighting_OccSensors',
 'LightingDay_EELighting_Garage_OccSensors',
 'ExteriorLightingDay',
 'LightingDay_EELighting',
 'RefrigeratorDay',
 'MiscPlugLoadDay',
 'CookingRangeDay',
 'DishwasherWeekday',
 'DishwasherWeekend',
 'DishwasherVacation',
 'ClothesWasherWeekday',
 'ClothesWasherWeekend',
 'ClothesWasherVacation',
 'dhw_profile_day',
 'ClothesDryerWeekday',
 'ClothesDryerWeekend',
 'ClothesDryerVacation',
 'SinksWeekday',
 'SinksWeekend',
 'SinksVacation',
 'ShowersWeekday',
 'ShowersWeekend',
 'ShowersVacation',
 'BathsWeekday',
 'BathsWeekend',
 'BathsVacation',
 'DHWDistDay',
 'WinterLoadDay',
 'SummerLoadDay',
 'RefrigeratorWeek',
 'MiscPlugLoadWeek',
 'CookingRangeWeek',
 'DishwasherWeek',
 'ClothesWasherWeek',
 'dhw_profile_week',
 'ClothesDryerWeek',
 'SinksWeek',
 'ShowersWeek',
 'BathsWeek',
 'DHWDistWeek',
 'OccupancyWeek',
 'LightingProfileWeek',
 'Li

Check if any used schedules are not prepared in the Schedule objects.

In [94]:
new_sch_needed = list(set(used_schedules) - set(existing_schedules))
new_sch_needed

['SWHSys1 Water Heater Setpoint Temperature Schedule Name',
 'SWHSys1 Water Heater Ambient Temperature Schedule Name',
 'PlantHPWHTempSch']

In [95]:
matching_objects = []

if len(new_sch_needed) > 0:    
    for sch in new_sch_needed:
    
    # Iterate through all objects in the source file
        for obj_type in sourceidf.idfobjects:  # Loop through all object types
            for obj in sourceidf.idfobjects[obj_type]:  # Loop through all objects of that type
                
                # Check if the schedule name appears in any field of the object
                if sch in obj.fieldvalues:
                    if "SCHEDULE" in obj_type: #Make sure only the Schedule object is included, not the object that uses the schedule
                        matching_objects.append(obj)
    
    #Copy the new schedules to the target file
    for new_obj in matching_objects:
        targetidf.copyidfobject(new_obj)

In [96]:
matching_objects

[
 Schedule:Compact,
     SWHSys1 Water Heater Setpoint Temperature Schedule Name,    !- Name
     Temperature,              !- Schedule Type Limits Name
     Through: 12/31,           !- Field 1
     For: AllDays,             !- Field 2
     Until: 24:00,             !- Field 3
     60.0;                     !- Field 4,
 
 Schedule:Compact,
     SWHSys1 Water Heater Ambient Temperature Schedule Name,    !- Name
     Temperature,              !- Schedule Type Limits Name
     Through: 12/31,           !- Field 1
     For: AllDays,             !- Field 2
     Until: 24:00,             !- Field 3
     22.0;                     !- Field 4,
 
 Schedule:Compact,
     PlantHPWHTempSch,         !- Name
     Any Number,               !- Schedule Type Limits Name
     Through: 12/31,           !- Field 1
     For: AllDays,             !- Field 2
     Until: 24:00,             !- Field 3
     48.89;                    !- Field 4]

# Save as

In [97]:
targetidf.saveas('US+MF+CZ2AWH+gasfurnace+slab+IECC_2006_HPWH.idf')